### Description

Fonctionnement d'un RNN élémentaire (2x1) il prend en entrée deux inputs et sort un output.

Backpropagation : calcul et ajustement des weights à chaque itération. On optimise pour réduire l'écart entre les valeurs prédites et les valeurs réelles.  

Ce fichier reproduit l'algo de backpropagation sur un RNN simple dont  l'architecture est la suivante : 
hidden layer : 2x2
output layer : 2x1

In [1]:
import tensorflow as tf
print("tf version is ", tf.__version__)

tf.keras.backend.set_floatx('float64') # on travaille avec des flottants uniquement

tf version is  2.9.1


### construction du modèle

In [19]:
class model(tf.keras.Model) :
    
    def __init__(self, x, y) :
        super(model, self).__init__()
        
        # input layer : deux inputs constants
        self.x_input = tf.constant(x, dtype = tf.float64)
        self.y_input = tf.constant(y, dtype = tf.float64) 
        
        # layer 1 - hidden layer
        self.w1 = tf.Variable(tf.random.uniform([2,2], 
                                                minval = 0., 
                                                maxval = 1., 
                                                dtype = tf.float64), 
                                               name = 'w1') # matrice des poids : 2 neuronnes - 2 poids par neuronne ; trainable = T ie utilisable dans la méthode du gradient descendant
        
        self.b1 = tf.Variable(tf.random.uniform([2], 
                                                minval = 0., 
                                                maxval = 1., 
                                                dtype = tf.float64), 
                                               name = 'b1') # vecteur de biais sur les poids 
        
        # layer 2 - output layer
        self.w2 = tf.Variable(tf.random.uniform([2,1], 
                                                minval = 0., 
                                                maxval = 1., 
                                                dtype = tf.float64), 
                                               name = 'w2') # matrice des poids : 2 neuronnes - 2 poids par neuronne ; trainable = T ie utilisable dans la méthode du gradient descendant
        
        self.b2 = tf.Variable(tf.random.uniform([1], 
                                                minval = 0., 
                                                maxval = 1., 
                                                dtype = tf.float64), 
                                               name = 'b2') # vecteur de biais sur les poids 
    def call(self, inputs) :
        # partie calculs
        inputs = tf.constant(x, dtype = tf.float64)
        
        in_neurons_hidden_layer = tf.add(tf.linalg.matmul(inputs, self.w1), self.b1) # x*w + biais
        out_neurons_hidden_layer = tf.sigmoid(in_neurons_hidden_layer) # on applique une fonction d'activation sigmoide au résultat 

        in_neurons_output_layer = tf.add(tf.linalg.matmul(out_neurons_hidden_layer, self.w2), self.b2) # prend en entrée l'output de la couche hidden précédente + fais le calcul
        out_neurons_output_layer = tf.sigmoid(in_neurons_output_layer) # on applique une fonction d'activation sigmoide au résultat 

        return out_neurons_output_layer

### training

In [5]:
import numpy as np
import pandas as pd
from datetime import datetime 

# XOR : illustration du fonctionnement du RNN

x = np.array([[0.,0.], [1.,1.], [0.,1.], [1.,0.]])
y = np.array([1.,0.,1.,1.])

pd.DataFrame({
    'input1' : np.vstack(x).T[0],
    'input2' : np.vstack(x).T[1],
    'output' : y
})

,input1,input2,output
0,0.0,0.0,1.0
1,1.0,1.0,0.0
2,0.0,1.0,1.0
3,1.0,0.0,1.0


In [20]:
# efface les logs des apprentissages passés
#!rm -rf ./logs/
# creation d'un fichier de récap pour tensorboard
current_time = datetime.now().strftime('%Y%m%d-%H%M%S')
logdir = 'logs/gradient_tape/'+ current_time + '/train'
summary_writer = tf.summary.create_file_writer(logdir)

# backpropagation phase :: descente du gradient 
# loss function : calcule le delta entre la pred du modele et la valeur attendue et optimise les poids pour réduire le delta
model = model(x, y)
optimizer = tf.optimizers.SGD(learning_rate = 0.1) # optimizer du (stochastic) gradient descent

def loss(outputs_model, targets) :
    # perte = ecart entre le target et l'output du modele 
    error = tf.math.subtract(targets, outputs_model)
    return tf.reduce_sum(tf.square(error)) # on choisit de retourner l'erreur quadratique ici par exemple


def get_gradient(model, inputs, targets) :
    
    with tf.GradientTape() as tape : 
        loss_value = loss(model(inputs), targets)
    
    return tape.gradient(loss_value, [model.w1, model.b1, model.w2, model.b2])


def run_network(inputs, targets, epochs) : 
    
    for i in range(epochs) :
        grads = get_gradient(model, inputs, targets)
        optimizer.apply_gradients(zip(grads, [model.w1, model.b1, model.w2, model.b2]))
        loss_epoch = loss(model(inputs), model.y_input)
        
        with summary_writer.as_default() :
           tf.summary.scalar('loss', loss_epoch, step=i)
        if i % 100 == 0 :
            print(f"Loss at the epoch {i} : {loss_epoch}.")

In [11]:
#help(zip)

In [21]:
# compilation avec la fonction run neetwork
run_network(x, y, epochs = 1000) 

Loss at the epoch 0 : 3.0327095116222766.
Loss at the epoch 100 : 3.002374584972741.
Loss at the epoch 200 : 3.0019573106929176.
Loss at the epoch 300 : 3.001632303538483.
Loss at the epoch 400 : 3.0013743135725877.
Loss at the epoch 500 : 3.0011662473654326.
Loss at the epoch 600 : 3.0009961819222744.
Loss at the epoch 700 : 3.000855584785963.
Loss at the epoch 800 : 3.0007382124621236.
Loss at the epoch 900 : 3.0006394062160773.


### tensorboad - post evaluation

In [15]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [22]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 6816), started 0:05:51 ago. (Use '!kill 6816' to kill it.)

In [17]:
import tensorboard as tb
print("tb version is ", tb.__version__)

tb version is  2.10.0a20220807


In [3]:
# combien de versions de tensorboard installées ?
import pkg_resources

for entry_point in pkg_resources.iter_entry_points('tensorboard_plugins'):
    print(entry_point.dist)


tensorboard-plugin-wit 1.8.1
tb-nightly 2.10.0a20220807
